In [120]:
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, models, callbacks, optimizers
from keras.utils import load_img, img_to_array
from keras.optimizers import RMSprop, SGD, Adam
from keras.applications import ResNet50, VGG16
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, AveragePooling2D
import re, sys, datetime, json, random, cv2, os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import keras
from PIL import Image
import cv2
import os
import shutil
import cv2
import numpy as np
import tensorflow as tf

In [121]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [122]:
!ls  "/content/gdrive/MyDrive/images"

all_snow  no_snow  output_folder  partial_snow


In [123]:
FAST_RUN = False
IMAGE_WIDTH=256
IMAGE_HEIGHT=256
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
INPUT_SHAPE=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)

NUM_CLASSES = 2 # dusty and clean
RESNET50_POOLING_AVERAGE = 'avg'
DENSE_LAYER_ACTIVATION = 'sigmoid'
OBJECTIVE_FUNCTION = 'binary_crossentropy'
LOSS_METRICS = ['accuracy']
STEPS_PER_EPOCH_TRAINING = 10
STEPS_PER_EPOCH_VALIDATION = 10

# EARLY_STOP_PATIENCE must be < NUM_EPOCHS
NUM_EPOCHS = 150
EARLY_STOP_PATIENCE = 3

# Define a list of possible batch sizes
BATCH_SIZES = [16, 32, 64, 128]
# Generate a random index to select a batch size from the list
random_index = np.random.randint(0, len(BATCH_SIZES))

# Select a batch size using the random index
BATCH_SIZE = BATCH_SIZES[random_index]


In [124]:
all_snow_dir = '/content/gdrive/MyDrive/images/all_snow'
no_snow_dir = '/content/gdrive/MyDrive/images/no_snow'
partial_snow_dir = '/content/gdrive/MyDrive/images/partial_snow'

# Output folder path for saving the transformed images
output_folder = '/content/gdrive/MyDrive/images/output_folder'

In [125]:
# Load pre-trained CNN model
model = tf.keras.applications.VGG16(weights='imagenet', include_top=False)

In [126]:
# convert images to Infrared images
def convert_to_infrared(image):
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    normalized_image = grayscale_image / 255.0
    color_mapped_image = (normalized_image * 255).astype('uint8')

    colormap = cv2.COLORMAP_JET
    infrared_image = cv2.applyColorMap(color_mapped_image, colormap)

    return infrared_image
    

# Function to process an image using the CNN model and apply night mode effect
def process_image(image_path):
    # Load the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Apply the infrared night vision effect
    infrared_image = convert_to_infrared(image)

    return infrared_image


# This function will apply the night mode effect to all images in a folder
def convert_to_night_mode(folder_path, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process each image in the folder
    for filename in os.listdir(folder_path):
        if filename.endswith('.jpg') or filename.endswith('.jpeg') or filename.endswith('.png'):
            image_path = os.path.join(folder_path, filename)
            output_path = os.path.join(output_folder, filename)
            print(output_path)

            try:
                # Process the image and save the transformed image
                transformed_image = process_image(image_path)

                # Save the transformed image
                cv2.imwrite(output_path, transformed_image)

            except Exception as e:
                print(f"Error processing image {image_path}: {str(e)}")

    print("Image transformation completed!")

def move_all_files(source_folder, destination_folder):
    # Get the list of files in the source folder
    file_list = os.listdir(source_folder)

    # Iterate over the file list and move each file to the destination folder
    for file_name in file_list:
        source_file = os.path.join(source_folder, file_name)
        destination_file = os.path.join(destination_folder, file_name)
        shutil.move(source_file, destination_file)

    print("Files moved successfully!")


In [131]:
# Folder path containing the images to be transformed
folder_path = no_snow_dir
# folder_path = all_snow_dir
# folder_path = partial_snow_dir

convert_to_night_mode(folder_path, output_folder)


/content/gdrive/MyDrive/images/output_folder/Copy of images (154).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of images (156).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of images (155).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of images (158).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of images (150).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of images (151).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of images (146).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of images - 2023-03-08T211748.357 (1).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of images (152).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of images (149).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of images (137).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of images (135).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of images (130).jpeg
/content/gdrive/MyDrive/images/output_folder/Copy of im

In [132]:
# after moved wait for at least 5 min as google drive takes times internally to move the files from one folder to another depends on sizes
destination_folder = '/content/gdrive/MyDrive/Snow_Covered_Solar_Panel_Detect/Images/train/no_snow'

move_all_files(output_folder, destination_folder)

Files moved successfully!
